# Removing Duplicate Ligands
Makes csv with unique compounds while assigning a unique ID for each 

In [139]:
import pandas as pd
import numpy as np 
import uuid

In [199]:
def remove_dupes(tsv1,tsv2):
    '''
        Create a new csv file with only the unique compounds from the input csvs
        input: two tsv files that have at least the following columns: 'compound_name' and 'SMILES'
        output: a csv file with only the unqiue compounds from both csvs along with a unique ID for each compound 
    '''
    # reading in both tsv 
    df1 = pd.read_csv(tsv1, sep='\t')[['compound_name', 'SMILES']]
    df2 = pd.read_csv(tsv2, sep='\t')[['compound_name', 'SMILES']]
    
    # dealing with unique SMILES
    df1_ligs = set(df1['SMILES'])
    df2_ligs = set(df2['SMILES'])

    # differences and intersections
    diff_df1 = df1_ligs-df2_ligs
    diff_df2 = df2_ligs-df1_ligs
    intersect = df2_ligs.intersection(df1_ligs)

    # dataframes with compound_name and SMILES info 
    diff_df1_df = df1[df1['SMILES'].isin(diff_df1)]
    diff_df2_df = df2[df2['SMILES'].isin(diff_df2)]
    intersect_df = df1[df1['SMILES'].isin(intersect)]
    
    # only unique compounds 
    final_merged_df = pd.concat([intersect_df, diff_df1_df, diff_df2_df], ignore_index=True)

    # adding unique identifiers for compound_names == NaN
    final_merged_df['uniq_ids'] = [str(uuid.uuid4()) for i in range(len(final_merged_df))]
    final_merged_df['compound_name'] = final_merged_df.apply(lambda row: row['uniq_ids'] if pd.isna(row['compound_name']) else row['compound_name'], axis=1)
    final_merged_df.set_index('uniq_ids', inplace=True)
    
    # writing to a new csv file 
    filename = tsv1.split('.')[0]
    final_merged_df.to_csv(f'{filename}_unique.csv')
    

In [200]:
remove_dupes('B3DB_meng_2021_regression.tsv', 'B3DB_meng_2021_classification.tsv')

In [214]:
# match SMILES in each original tsv with made unqiuq id csv 
unique_ligs  = pd.read_csv('B3DB_meng_2021_regression_unique.csv')[['uniq_ids', 'SMILES']]
regress_ligs = pd.read_csv('B3DB_meng_2021_regression.tsv', sep='\t')
classification_ligs = pd.read_csv('B3DB_meng_2021_classification.tsv', sep='\t')


regress_merged = pd.merge(unique_ligs, regress_ligs, on='SMILES')
classification_merged = pd.merge(unique_ligs, classification_ligs, on='SMILES')

classification_merged.to_csv('B3DB_classification_uniq_ids.csv')
regress_merged.to_csv('B3DB_regression_uniq_ids.csv')

---------------------------------------------
### Testing below


In [6]:
# smiles in regression file 
meng_reg = pd.read_csv('B3DB_meng_2021_regression.tsv', sep='\t')[['compound_name', 'SMILES']]
meng_reg

,compound_name,SMILES
0,moxalactam,CN1C(=NN=N1)SCC2=C(N3C(C(C3=O)(NC(=O)C(C4=CC=C...
1,schembl614298,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)OC6[C@...
2,morphine-6-glucuronide,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)O)O[C@...
3,2-[4-(5-bromo-3-methylpyridin-2-yl)butylamino]...,CC1=NC=C(C=C1)CC2CNC(NC2=O)NCCCCC3=NC=C(C=C3C)Br
4,NaN,c1(c2c3n(c4c(C(N(C)C3)=O)c(Cl)ccc4)cn2)noc(C(O...
...,...,...
1053,2qju,C[NH2+]CCCN1C2=CC=CC=C2CCC3=CC=CC=C31
1054,promazine,CN(C)CCCN1C2=CC=CC=C2SC3=CC=CC=C31
1055,diphenhydramine,CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2
1056,1225-64-5,CNCCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl


In [117]:
# compounds that have NaN value
meng_reg.isna().sum()

compound_name    22
SMILES            0
dtype: int64

In [7]:
# smiles in classification file 
meng_classi = pd.read_csv('B3DB_meng_2021_classification.tsv', sep='\t')[['compound_name', 'SMILES']]
meng_classi

,compound_name,SMILES
0,sulphasalazine,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O
1,moxalactam,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...
2,clioquinol,Oc1c(I)cc(Cl)c2cccnc12
3,bbcpd11 (cimetidine analog) (y-g13),CCNC(=NCCSCc1ncccc1Br)NC#N
4,schembl614298,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...
...,...,...
7802,antazoline,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1
7803,emedastine,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21
7804,ketotifen,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1
7805,milrinone,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1


In [119]:
meng_classi.isna().sum()

compound_name    1109
SMILES              0
dtype: int64

In [50]:
# finding difference between the two by SMILES
reg_ligs = set(meng_reg['SMILES'])
classi_ligs = set(meng_classi['SMILES'])

In [99]:
# different molecules in classi_ligs
diff_classi = classi_ligs-reg_ligs
len(diff_classi)

7744

In [98]:
# different molecules in reg_ligs
diff_reg = reg_ligs-classi_ligs
len(diff_reg)

995

In [104]:
# finding the intersection between the two 
intersec = classi_ligs.intersection(reg_ligs)
len(intersec)
# 958

63

In [101]:
diff_classi_df = meng_classi[meng_classi['SMILES'].isin(diff_classi)]
diff_classi_df

,compound_name,SMILES
0,sulphasalazine,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O
1,moxalactam,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...
2,clioquinol,Oc1c(I)cc(Cl)c2cccnc12
3,bbcpd11 (cimetidine analog) (y-g13),CCNC(=NCCSCc1ncccc1Br)NC#N
4,schembl614298,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...
...,...,...
7802,antazoline,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1
7803,emedastine,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21
7804,ketotifen,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1
7805,milrinone,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1


In [103]:
diff_reg_df = meng_reg[meng_reg['SMILES'].isin(diff_reg)]
diff_reg_df

,compound_name,SMILES
0,moxalactam,CN1C(=NN=N1)SCC2=C(N3C(C(C3=O)(NC(=O)C(C4=CC=C...
1,schembl614298,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)OC6[C@...
2,morphine-6-glucuronide,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)O)O[C@...
3,2-[4-(5-bromo-3-methylpyridin-2-yl)butylamino]...,CC1=NC=C(C=C1)CC2CNC(NC2=O)NCCCCC3=NC=C(C=C3C)Br
4,NaN,c1(c2c3n(c4c(C(N(C)C3)=O)c(Cl)ccc4)cn2)noc(C(O...
...,...,...
1053,2qju,C[NH2+]CCCN1C2=CC=CC=C2CCC3=CC=CC=C31
1054,promazine,CN(C)CCCN1C2=CC=CC=C2SC3=CC=CC=C31
1055,diphenhydramine,CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2
1056,1225-64-5,CNCCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl


In [105]:
# doesnt matter which df we get this from 
intersect_df = meng_reg[meng_reg['SMILES'].isin(intersec)]
intersect_df

,compound_name,SMILES
133,"nitrous oxide, 99%",N#[N+]O
219,(-)-3-methylhexane,CCC[C@H](C)CC
260,telmesteine,CCOC(=O)N1CSCC1C(=O)O
398,s-methylisothiourea,CSC(=N)N
443,acrylonitrile,C=CC#N
...,...,...
1037,pentane,CCCCC
1038,n-hexane,CCCCCC
1040,2-methylheptane,CCCCCC(C)C
1045,cyclohexane,C1CCCCC1


In [181]:
merge_meng_df = pd.concat([intersect_df, diff_reg_df, diff_classi_df], ignore_index=True)
merge_meng_df

,compound_name,SMILES
0,"nitrous oxide, 99%",N#[N+]O
1,(-)-3-methylhexane,CCC[C@H](C)CC
2,telmesteine,CCOC(=O)N1CSCC1C(=O)O
3,s-methylisothiourea,CSC(=N)N
4,acrylonitrile,C=CC#N
...,...,...
8797,antazoline,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1
8798,emedastine,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21
8799,ketotifen,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1
8800,milrinone,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1


In [182]:
merge_meng_df.isna().sum().iloc[0]

1131

In [191]:
# just making index a unique id col 
import uuid
merge_meng_df['uniq_ids'] = [str(uuid.uuid4()) for i in range(len(merge_meng_df))]
merge_meng_df['compound_name'] = merge_meng_df.apply(lambda row: row['uniq_ids'] if pd.isna(row['compound_name']) else row['compound_name'], axis=1)
merge_meng_df.set_index('uniq_ids', inplace=True)
merge_meng_df

,compound_name,SMILES
uniq_ids,,
f3d00b62-e7ce-4cf6-a534-bc076ddd6e4d,"nitrous oxide, 99%",N#[N+]O
f66f77bd-2658-4b31-83f1-0b633aabe783,(-)-3-methylhexane,CCC[C@H](C)CC
a2ec11b7-f984-41e6-a629-6451d313ed42,telmesteine,CCOC(=O)N1CSCC1C(=O)O
3f0b7942-45ae-4d63-a2c9-442cb3223de8,s-methylisothiourea,CSC(=N)N
0671eb6e-35cf-4e83-a52d-fd4a3f9ec9ad,acrylonitrile,C=CC#N
...,...,...
d6077e0b-996c-4dc5-a264-d9fa535daf7c,antazoline,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1
56fcca80-2a0d-40b1-8c10-1538a3e6b296,emedastine,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21
b3288ddb-3661-46c9-a5ed-eff2a4331b29,ketotifen,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1


In [151]:
# want to assign unique ids to each compound 

# uniq_ids = [str(uuid.uuid4()) for i in range(merge_meng_df.isna().sum().iloc[0])]
# unique_id_series = pd.Series(uniq_ids, index=merge_meng_df.index[merge_meng_df['compound_name'].isna()])
# print(uniq_ids)

# merge_meng_df['compound_name'] = merge_meng_df['compound_name'].combine_first(unique_id_series)
    
# merge_meng_df.isna().sum()

['4ae95180-2725-437e-bb7a-63f947fc79bb', '8e432eea-8f3c-4880-956f-7eb62d52f8a3', 'ffa9a8f3-2302-428f-989b-c914a1c74a46', '4b9e7732-b231-4b49-8cd1-39e48dd2a64f', 'dab39f3c-2d4d-4468-96db-5be3469b4e92', '996b2e26-d5cb-4c78-9b52-edb220d2cded', 'dc3582b2-a523-43e7-abc9-a279e66a86af', '1c693bc9-0c1a-44a0-8abd-0cb12a06e0b4', '573e526e-6437-45e9-b22b-a9cf87e4a262', '52631ae1-3b93-4542-ab24-696cbbb3445e', '14598288-8f0c-44d8-89c1-94c0a985b2aa', '84c50b3b-0a24-435e-909e-2635274cf7bd', '8b593e2a-ba95-44c5-bfc0-7cef13143d13', 'ad190cc4-a061-4fbe-a515-372bbb34038f', '8535444d-a06e-4aba-889e-80ee98efa8b0', '9442e1a1-1052-4e77-90c5-1c280153a099', '0c8487f4-4439-447d-8088-7a1f8e34e28f', '2b00f629-55db-404c-8cda-792ba4d379b9', '8fef1c22-b0c0-46e9-805b-1e19196cd34f', '175982de-ee3d-47ff-839d-ac0b767ee6aa', '99b3798c-d4cf-46f7-9182-bbc101f5f167', '966b4671-0c5f-4bf0-8313-762e5644733c', 'b492f524-4771-41ef-b5ce-34ba53c037d9', '3b9837d6-c6f3-47f6-88d1-98783b111848', '9ce55453-7535-4805-97b3-0200a40b3659',

compound_name    0
SMILES           0
dtype: int64

In [192]:
# write to new csv 
merge_meng_df.to_csv('meng_unique.csv')


In [82]:
# POC
df1 = pd.DataFrame({'ID': ['A', 'B', 'C'], 'Value': [1, 2, 3]})
df2 = pd.DataFrame({'ID': ['A', 'D', 'E', 'F'], 'Value': [1, 4, 5, 6]})

In [83]:

set1 = set(df1['Value'])
set2 = set(df2['Value'])

diff = set1-set2
# only gives things in set 1 that are not in set2
diff


{2, 3}

In [84]:
diff2 = set2-set1
# onyly gives things in set 2 that are not in set1
diff2

{4, 5, 6}

In [85]:
intersect = set1.intersection(set2)
# only gives the ones that are same between the two 
intersect

{1}

In [86]:
# we want rows of intersect, diff, are diff2 in final df
diff_df1 = df1[df1['Value'].isin(diff)]
diff_df1

,ID,Value
1,B,2
2,C,3


In [87]:
diff_df2 = df2[df2['Value'].isin(diff2)]
diff_df2

,ID,Value
1,D,4
2,E,5
3,F,6


In [88]:
intersect_df = df1[df1['Value'].isin(intersect)]
intersect_df

,ID,Value
0,A,1


In [89]:
# create combined df 
result_df = pd.concat([diff_df1, diff_df2, intersect_df],ignore_index=True)
result_df

,ID,Value
0,B,2
1,C,3
2,D,4
3,E,5
4,F,6
5,A,1
